<h1 align="center"> <span style="font-size: 20px;"> Proyecto Final Grupo Nº 7 </span> <br>
    <align="center"> <span style="font-size: 15px;">  Proyecto Yelp & Google Maps - Reviewa y Recomendaciones</span><br>         
<align="center"> <span style="font-size: 15px;">@utores:  Javier Castro, Luca Ramallo, Luis Ramírez, Lesmen García.</span> <br>
  <a href="https://github.com/lesmengp/Proyecto-Final-Grupo-07.git">GitHub: <span style="font-size: 20px;">Proyecto Yelp & Google Maps</span></a> 
</h1>

# Consolidando Datasets de Business, Review (6990280) y Tip para el Modelo de ML

### Librerías a Utilizar:

In [2]:
!pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 10.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.1 MB/s eta 0:00:00
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/8f/3e/4b8b40eb3c80aeaf360f0361d956d129bb3d23b2a3ecbe3a04a8f3bdd6d3/regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 49.0 MB/s eta 0:00:00


In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io
import json
import gcsfs
import ast
from textblob import TextBlob   #0.17.1
import pandas as pd
import folium
import webbrowser
import textwrap
from folium.plugins import MarkerCluster

### Definición de Funciones

In [10]:
#############################################
### Funcion para Visualizar Valores Nulos ###
#############################################

def ValoresNulos(columnas,datasets):
    for dataset in datasets:
        df = globals()[dataset]
        print(f"Información del DataSets: {datasets}")
        print("="*80)
        for columna in columnas:
            print(f"Analizando los Valores Nulos en la Columna:   '{columna}'")
            print(f"Total Registros: {df.shape[0]}")
            print(f"Total Valores Nulos: {df[columna].isna().sum()}")
            print(f"Porcentaje Valores Nulos: {round((df[columna].isna().sum()/df.shape[0])*100,3)} %")
            print("="*80)

In [11]:
#####################################################
### Funcion para Eliminar Filas con Valores Nulos ###
#####################################################

def EliminarRegistrosValoresNulos(columnas,datasets):
    for dataset in datasets:
        for columna in columnas:
            df = globals()[dataset]
            df.dropna(subset=columna, inplace=True)
    df.info()

In [12]:
#################################################
### Funcion para Visualizar Columnas Anidadas ###
#################################################

def VisualizarColumnas(columnas,datasets):
    pd.set_option('display.max_colwidth', None)
    for dataset in datasets:
        for columna in columnas:
            df = globals()[dataset]
            print(f"{df[columna]}\n")
            print(f"Tipo de Columna:, {type(df[columna])}")
            print("="*80)
    pd.reset_option('display.max_colwidth')

### Cargando Dataset Consolidado: 'df_business_yelp_ML.parquet'

In [6]:
### Cargando DataSets Consolidado para Análisis de ML: 'df_business_yelp_ML.parquet'
# Ruta completa en Cloud Storage
ruta_eda_datalike = "gs://gmy/eda/df_business_yelp_ML.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
df_business_yelp_ML = pd.read_parquet(ruta_eda_datalike, storage_options={"project": "Proyecto Final - Henry"})
df_business_yelp_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49640 entries, 0 to 49639
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   business_id                 49640 non-null  object 
 1   name                        49640 non-null  object 
 2   address                     49640 non-null  object 
 3   city                        49640 non-null  object 
 4   state                       49640 non-null  object 
 5   latitude                    49640 non-null  float64
 6   longitude                   49640 non-null  float64
 7   stars                       49640 non-null  float64
 8   review_count                49640 non-null  int64  
 9   Categoria1                  49629 non-null  object 
 10  SubCategorias               49640 non-null  object 
 11  RestaurantsTakeOut          46181 non-null  object 
 12  BusinessAcceptsCreditCards  44033 non-null  object 
 13  RestaurantsDelivery         430

In [7]:
### Renombrando Columna 
df_business_yelp_ML = df_business_yelp_ML.rename(columns={'Categoria1':'Categorias'})

df_business_yelp_ML.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'latitude',
       'longitude', 'stars', 'review_count', 'Categorias', 'SubCategorias',
       'RestaurantsTakeOut', 'BusinessAcceptsCreditCards',
       'RestaurantsDelivery', 'OutdoorSeating', 'BikeParking',
       'RestaurantsReservations', 'WiFi', 'HasTV', 'GoodForKids',
       'RestaurantsGoodForGroups', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Saturday', 'Sunday'],
      dtype='object')

In [ ]:
### Cargando DataSets Consolidado para Análisis de ML: 'df_reviews_yelp_totales_ML.parquet'
# Ruta completa en Cloud Storage
#ruta_eda_datalike = "gs://gmy/eda/yelp_reviews.parquet" ##gs://gmy/Yelp/review.json

ruta_eda_datalike = "gs://gmy/Yelp/review.json" ##gs://gmy/Yelp/review.json

# Lee el archivo Parquet directamente en un DataFrame de pandas
df_reviews_yelp = pd.read_json(ruta_eda_datalike, storage_options={"project": "Proyecto Final - Henry"})
df_reviews_yelp.info()

# Escenarios de Totalizacion y Promedios

### Totales de Estrellas agrupado por 'business_id'

In [57]:
total_estrellas = df_reviews_yelp.groupby(['business_id', 'stars'])['stars'].count().unstack(fill_value=0)
total_estrellas

stars,1,2,3,4,5
business_id,,,,,
--0iUa4sNDFiZFrAdIWhZQ,0,0,0,0,1
--7jw19RH9JKXgFohspgQw,1,0,0,0,0
--9osgUCSDUWUkoTLdvYhQ,0,0,0,1,3
--ARBQr1WMsTWiwOKOj-FQ,0,0,0,0,2
--ZVrH2X2QXBFdCilbirsw,0,0,0,0,1
...,...,...,...,...,...
zzg-Il9zxsaVXlCDrcG7hg,0,0,0,0,3
zzjFdJwXuxBOGe9JeY_EMw,4,1,1,6,18
zznJox6-nmXlGYNWgTDwQQ,4,0,0,0,0


In [58]:
total_estrellas['Total_Estrellas'] = total_estrellas.sum(axis=1)
total_estrellas

stars,1,2,3,4,5,Total_Estrellas
business_id,,,,,,
--0iUa4sNDFiZFrAdIWhZQ,0,0,0,0,1,1
--7jw19RH9JKXgFohspgQw,1,0,0,0,0,1
--9osgUCSDUWUkoTLdvYhQ,0,0,0,1,3,4
--ARBQr1WMsTWiwOKOj-FQ,0,0,0,0,2,2
--ZVrH2X2QXBFdCilbirsw,0,0,0,0,1,1
...,...,...,...,...,...,...
zzg-Il9zxsaVXlCDrcG7hg,0,0,0,0,3,3
zzjFdJwXuxBOGe9JeY_EMw,4,1,1,6,18,30
zznJox6-nmXlGYNWgTDwQQ,4,0,0,0,0,4


### Renombrando las Columnas

In [59]:
### Renombrando las Columnas
nombres_estrellas = {
    1: 'Total_UnaEstrella',
    2: 'Total_DosEstrellas',
    3: 'Total_TresEstrellas',
    4: 'Total_CuatroEstrellas',
    5: 'Total_CincoEstrellas'
}

total_estrellas.rename(columns=nombres_estrellas, inplace=True)
total_estrellas

stars,Total_UnaEstrella,Total_DosEstrellas,Total_TresEstrellas,Total_CuatroEstrellas,Total_CincoEstrellas,Total_Estrellas
business_id,,,,,,
--0iUa4sNDFiZFrAdIWhZQ,0,0,0,0,1,1
--7jw19RH9JKXgFohspgQw,1,0,0,0,0,1
--9osgUCSDUWUkoTLdvYhQ,0,0,0,1,3,4
--ARBQr1WMsTWiwOKOj-FQ,0,0,0,0,2,2
--ZVrH2X2QXBFdCilbirsw,0,0,0,0,1,1
...,...,...,...,...,...,...
zzg-Il9zxsaVXlCDrcG7hg,0,0,0,0,3,3
zzjFdJwXuxBOGe9JeY_EMw,4,1,1,6,18,30
zznJox6-nmXlGYNWgTDwQQ,4,0,0,0,0,4


In [ ]:
# Agrupa el DataFrame original por 'business_id' y calcula el ptotal de estrellas
#df_total_stars_review = df_reviews_yelp.groupby('business_id')['stars'].sum().reset_index()

# Renombra la columna de promedio de estrellas
#df_total_stars_review.columns = ['business_id', 'total_stars']
#df_total_stars_review

### # Agrupa el DataFrame por 'business_id' y calcula la suma de sentimientos positivos, neutros y negativos

In [60]:
df_reviews_yelp['sentimientoReviewTextBlob'].value_counts()

sentimientoReviewTextBlob
 1    596805
-1     97558
 0      5637
Name: count, dtype: int64

In [61]:
# Agrupa el DataFrame por 'business_id' y calcula la suma de sentimientos positivos, neutros y negativos
sentimientos_por_negocio_total = df_reviews_yelp.groupby('business_id')['sentimientoReviewTextBlob'].agg(
    total_sentimientos='count',  # Total de sentimientos
    sentimientos_positivos=lambda x: (x == 1).sum(),  # Total de sentimientos positivos
    sentimientos_neutros=lambda x: (x == 0).sum(),  # Total de sentimientos neutros
    sentimientos_negativos=lambda x: (x == -1).sum(),  # Total de sentimientos negativos
).reset_index()

# Resultado
sentimientos_por_negocio_total

,business_id,total_sentimientos,sentimientos_positivos,sentimientos_neutros,sentimientos_negativos
0,--0iUa4sNDFiZFrAdIWhZQ,1,1,0,0
1,--7jw19RH9JKXgFohspgQw,1,1,0,0
2,--9osgUCSDUWUkoTLdvYhQ,4,4,0,0
3,--ARBQr1WMsTWiwOKOj-FQ,2,2,0,0
4,--ZVrH2X2QXBFdCilbirsw,1,0,0,1
...,...,...,...,...,...
40187,zzg-Il9zxsaVXlCDrcG7hg,3,3,0,0
40188,zzjFdJwXuxBOGe9JeY_EMw,30,25,0,5
40189,zznJox6-nmXlGYNWgTDwQQ,4,1,0,3
40190,zznZqH9CiAznbkV6fXyHWA,12,12,0,0


### Uniendo DataFrame de Estrellas y Sentimiento por Negocio

In [62]:
### Uniendo DataFrame de Estrellas y Sentimiento por Negocio

# Utiliza la función merge para concatenar los DataFrames por 'business_id'
result_df = total_estrellas.merge(sentimientos_por_negocio_total, on='business_id')

# Resultado
result_df

,business_id,Total_UnaEstrella,Total_DosEstrellas,Total_TresEstrellas,Total_CuatroEstrellas,Total_CincoEstrellas,Total_Estrellas,total_sentimientos,sentimientos_positivos,sentimientos_neutros,sentimientos_negativos
0,--0iUa4sNDFiZFrAdIWhZQ,0,0,0,0,1,1,1,1,0,0
1,--7jw19RH9JKXgFohspgQw,1,0,0,0,0,1,1,1,0,0
2,--9osgUCSDUWUkoTLdvYhQ,0,0,0,1,3,4,4,4,0,0
3,--ARBQr1WMsTWiwOKOj-FQ,0,0,0,0,2,2,2,2,0,0
4,--ZVrH2X2QXBFdCilbirsw,0,0,0,0,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
40187,zzg-Il9zxsaVXlCDrcG7hg,0,0,0,0,3,3,3,3,0,0
40188,zzjFdJwXuxBOGe9JeY_EMw,4,1,1,6,18,30,30,25,0,5
40189,zznJox6-nmXlGYNWgTDwQQ,4,0,0,0,0,4,4,1,0,3
40190,zznZqH9CiAznbkV6fXyHWA,0,0,0,1,11,12,12,12,0,0


In [63]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40192 entries, 0 to 40191
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   business_id             40192 non-null  object
 1   Total_UnaEstrella       40192 non-null  int64 
 2   Total_DosEstrellas      40192 non-null  int64 
 3   Total_TresEstrellas     40192 non-null  int64 
 4   Total_CuatroEstrellas   40192 non-null  int64 
 5   Total_CincoEstrellas    40192 non-null  int64 
 6   Total_Estrellas         40192 non-null  int64 
 7   total_sentimientos      40192 non-null  int64 
 8   sentimientos_positivos  40192 non-null  int64 
 9   sentimientos_neutros    40192 non-null  int64 
 10  sentimientos_negativos  40192 non-null  int64 
dtypes: int64(10), object(1)
memory usage: 3.4+ MB


### Totalizando por 'useful', 'funny' y 'cool'

In [64]:
### Totalizando por 'useful', 'funny' y 'cool'

# Agrupa el DataFrame por 'business_id' y calcula la suma de 'useful', 'funny' y 'cool'
totals_by_business = df_reviews_yelp.groupby('business_id').agg({
    'useful': 'sum',  # Suma de 'useful' por negocio
    'funny': 'sum',   # Suma de 'funny' por negocio
    'cool': 'sum'     # Suma de 'cool' por negocio
}).reset_index()

# Resultado
totals_by_business

,business_id,useful,funny,cool
0,--0iUa4sNDFiZFrAdIWhZQ,0,0,0
1,--7jw19RH9JKXgFohspgQw,0,0,0
2,--9osgUCSDUWUkoTLdvYhQ,0,0,0
3,--ARBQr1WMsTWiwOKOj-FQ,0,0,0
4,--ZVrH2X2QXBFdCilbirsw,0,0,0
...,...,...,...,...
40187,zzg-Il9zxsaVXlCDrcG7hg,1,0,3
40188,zzjFdJwXuxBOGe9JeY_EMw,25,3,9
40189,zznJox6-nmXlGYNWgTDwQQ,1,0,0
40190,zznZqH9CiAznbkV6fXyHWA,23,3,17


### Renombrando Columnas

In [65]:
### Renombrando Columnas
nuevos_nombres = {
    'useful': 'Total_useful',
    'funny': 'Total_funny',
    'cool': 'Total_cool'
}
# Utiliza el método rename para renombrar las columnas
totals_by_business.rename(columns=nuevos_nombres, inplace=True)
totals_by_business

,business_id,Total_useful,Total_funny,Total_cool
0,--0iUa4sNDFiZFrAdIWhZQ,0,0,0
1,--7jw19RH9JKXgFohspgQw,0,0,0
2,--9osgUCSDUWUkoTLdvYhQ,0,0,0
3,--ARBQr1WMsTWiwOKOj-FQ,0,0,0
4,--ZVrH2X2QXBFdCilbirsw,0,0,0
...,...,...,...,...
40187,zzg-Il9zxsaVXlCDrcG7hg,1,0,3
40188,zzjFdJwXuxBOGe9JeY_EMw,25,3,9
40189,zznJox6-nmXlGYNWgTDwQQ,1,0,0
40190,zznZqH9CiAznbkV6fXyHWA,23,3,17


### Uniendo Estrellas, Sentimientos y 'useful', 'funny' y 'cool'. Todos por Totales

In [66]:
### Uniendo Estrellas, Sentimientos y 'useful', 'funny' y 'cool'. Todos por Totales

# Utiliza la función merge para concatenar los DataFrames por 'business_id'
result_df_totales = result_df.merge(totals_by_business, on='business_id')

# Resultado
result_df_totales

,business_id,Total_UnaEstrella,Total_DosEstrellas,Total_TresEstrellas,Total_CuatroEstrellas,Total_CincoEstrellas,Total_Estrellas,total_sentimientos,sentimientos_positivos,sentimientos_neutros,sentimientos_negativos,Total_useful,Total_funny,Total_cool
0,--0iUa4sNDFiZFrAdIWhZQ,0,0,0,0,1,1,1,1,0,0,0,0,0
1,--7jw19RH9JKXgFohspgQw,1,0,0,0,0,1,1,1,0,0,0,0,0
2,--9osgUCSDUWUkoTLdvYhQ,0,0,0,1,3,4,4,4,0,0,0,0,0
3,--ARBQr1WMsTWiwOKOj-FQ,0,0,0,0,2,2,2,2,0,0,0,0,0
4,--ZVrH2X2QXBFdCilbirsw,0,0,0,0,1,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40187,zzg-Il9zxsaVXlCDrcG7hg,0,0,0,0,3,3,3,3,0,0,1,0,3
40188,zzjFdJwXuxBOGe9JeY_EMw,4,1,1,6,18,30,30,25,0,5,25,3,9
40189,zznJox6-nmXlGYNWgTDwQQ,4,0,0,0,0,4,4,1,0,3,1,0,0
40190,zznZqH9CiAznbkV6fXyHWA,0,0,0,1,11,12,12,12,0,0,23,3,17


In [67]:
result_df_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40192 entries, 0 to 40191
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   business_id             40192 non-null  object
 1   Total_UnaEstrella       40192 non-null  int64 
 2   Total_DosEstrellas      40192 non-null  int64 
 3   Total_TresEstrellas     40192 non-null  int64 
 4   Total_CuatroEstrellas   40192 non-null  int64 
 5   Total_CincoEstrellas    40192 non-null  int64 
 6   Total_Estrellas         40192 non-null  int64 
 7   total_sentimientos      40192 non-null  int64 
 8   sentimientos_positivos  40192 non-null  int64 
 9   sentimientos_neutros    40192 non-null  int64 
 10  sentimientos_negativos  40192 non-null  int64 
 11  Total_useful            40192 non-null  int64 
 12  Total_funny             40192 non-null  int64 
 13  Total_cool              40192 non-null  int64 
dtypes: int64(13), object(1)
memory usage: 4.3+ MB


In [ ]:
### Renombrando Columnas
nuevos_nombres = {
    'sentimientos_positivos': 'Total_Sentimientos_Positivos',
    'sentimientos_neutros': 'Total_Sentimientos_Neutros',
    'sentimientos_negativos': 'Total_Sentimientos_Negativos',
    'total_sentimientos': 'Total_Sentimientos'
}
# Utiliza el método rename para renombrar las columnas
result_df_totales.rename(columns=nuevos_nombres, inplace=True)


In [70]:
result_df_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40192 entries, 0 to 40191
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   business_id                   40192 non-null  object
 1   Total_UnaEstrella             40192 non-null  int64 
 2   Total_DosEstrellas            40192 non-null  int64 
 3   Total_TresEstrellas           40192 non-null  int64 
 4   Total_CuatroEstrellas         40192 non-null  int64 
 5   Total_CincoEstrellas          40192 non-null  int64 
 6   Total_Estrellas               40192 non-null  int64 
 7   Total_Sentimientos            40192 non-null  int64 
 8   Total_Sentimientos_Positivos  40192 non-null  int64 
 9   Total_Sentimientos_Neutros    40192 non-null  int64 
 10  Total_Sentimientos_Negativos  40192 non-null  int64 
 11  Total_useful                  40192 non-null  int64 
 12  Total_funny                   40192 non-null  int64 
 13  Total_cool      

In [71]:
result_df_totales.head(3)

,business_id,Total_UnaEstrella,Total_DosEstrellas,Total_TresEstrellas,Total_CuatroEstrellas,Total_CincoEstrellas,Total_Estrellas,Total_Sentimientos,Total_Sentimientos_Positivos,Total_Sentimientos_Neutros,Total_Sentimientos_Negativos,Total_useful,Total_funny,Total_cool
0,--0iUa4sNDFiZFrAdIWhZQ,0,0,0,0,1,1,1,1,0,0,0,0,0
1,--7jw19RH9JKXgFohspgQw,1,0,0,0,0,1,1,1,0,0,0,0,0
2,--9osgUCSDUWUkoTLdvYhQ,0,0,0,1,3,4,4,4,0,0,0,0,0


In [86]:
df_reviews_totales = result_df_totales.copy()

### Cargando DataSets 'yelp_tips.parquet'

In [72]:
### Cargando DataSets 'yelp_tips.parquet'
# Ruta completa en Cloud Storage
ruta_eda_datalike = "gs://gmy/eda/yelp_tips.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
yelp_tips = pd.read_parquet(ruta_eda_datalike, storage_options={"project": "Proyecto Final - Henry"})
yelp_tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193030 entries, 0 to 193029
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   user_id                  193030 non-null  object        
 1   business_id              193030 non-null  object        
 2   text                     193030 non-null  object        
 3   date                     193030 non-null  datetime64[ns]
 4   compliment_count         193030 non-null  int64         
 5   sentimientoTips          193030 non-null  object        
 6   sentimientoTipsTextBlob  193030 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 10.3+ MB


In [74]:
yelp_tips['sentimientoTipsTextBlob'].value_counts()

sentimientoTipsTextBlob
1     137926
0      32622
-1     22482
Name: count, dtype: int64

In [80]:
yelp_tips['sentimientoTipsTextBlob'] = pd.to_numeric(yelp_tips['sentimientoTipsTextBlob'], errors='coerce')
yelp_tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193030 entries, 0 to 193029
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   user_id                  193030 non-null  object        
 1   business_id              193030 non-null  object        
 2   text                     193030 non-null  object        
 3   date                     193030 non-null  datetime64[ns]
 4   compliment_count         193030 non-null  int64         
 5   sentimientoTips          193030 non-null  object        
 6   sentimientoTipsTextBlob  193030 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 10.3+ MB


In [81]:
# Agrupa el DataFrame por 'business_id' y calcula la suma de sentimientos positivos, neutros y negativos
sentimientos_por_negocio_total = yelp_tips.groupby('business_id')['sentimientoTipsTextBlob'].agg(
    total_sentimientos='count',  # Total de sentimientos
    sentimientos_positivos=lambda x: (x == 1).sum(),  # Total de sentimientos positivos
    sentimientos_neutros=lambda x: (x == 0).sum(),  # Total de sentimientos neutros
    sentimientos_negativos=lambda x: (x == -1).sum(),  # Total de sentimientos negativos
).reset_index()

# Resultado
sentimientos_por_negocio_total

,business_id,total_sentimientos,sentimientos_positivos,sentimientos_neutros,sentimientos_negativos
0,---kPU91CF4Lq2-WlRu9Lw,4,4,0,0
1,--9osgUCSDUWUkoTLdvYhQ,1,1,0,0
2,--FcbSxK1AoEtEAxOgBaCw,3,1,1,1
3,--LC8cIrALInl2vyo701tg,2,1,1,0
4,--O3ip9NpXTKD4oBS1pY2A,2,0,0,2
...,...,...,...,...,...
55894,zzbZtgPYZS8sTIWQH6DwEw,3,1,0,2
55895,zzfj1-iPfw0cwnOjY0yUgA,1,0,1,0
55896,zzg-Il9zxsaVXlCDrcG7hg,1,1,0,0
55897,zzjFdJwXuxBOGe9JeY_EMw,1,1,0,0


In [83]:
sentimientos_por_negocio_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55899 entries, 0 to 55898
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   business_id             55899 non-null  object
 1   total_sentimientos      55899 non-null  int64 
 2   sentimientos_positivos  55899 non-null  int64 
 3   sentimientos_neutros    55899 non-null  int64 
 4   sentimientos_negativos  55899 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 2.1+ MB


### Renombrando Columnas

In [84]:
### Renombrando Columnas
nuevos_nombres = {
    'sentimientos_positivos': 'Total_SentimientosTip_Positivos',
    'sentimientos_neutros': 'Total_SentimientosTip_Neutros',
    'sentimientos_negativos': 'Total_SentimientosTip_Negativos',
    'total_sentimientos': 'Total_SentimientosTip'
}
# Utiliza el método rename para renombrar las columnas
sentimientos_por_negocio_total.rename(columns=nuevos_nombres, inplace=True)
sentimientos_por_negocio_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55899 entries, 0 to 55898
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   business_id                      55899 non-null  object
 1   Total_SentimientosTip            55899 non-null  int64 
 2   Total_SentimientosTip_Positivos  55899 non-null  int64 
 3   Total_SentimientosTip_Neutros    55899 non-null  int64 
 4   Total_SentimientosTip_Negativos  55899 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 2.1+ MB


In [85]:
df_tip_totales = sentimientos_por_negocio_total.copy()

### 1) Consolidando Datasets por Promedios

In [89]:
print(f"Dimensiones Dataset df_business: {df_business_yelp_ML.shape}")
print(f"Dimensiones Dataset df_reviews_totales: {df_reviews_totales.shape}")
print(f"Dimensiones Dataset df_tip_totales: {df_tip_totales.shape}")

Dimensiones Dataset df_business: (49640, 28)
Dimensiones Dataset df_reviews_totales: (40192, 14)
Dimensiones Dataset df_tip_totales: (55899, 5)


### Uniendo DataFrame de df_business y df_reviews por Negocio (Totales)

In [90]:
### Uniendo DataFrame de df_business y df_reviews por Negocio (Promedios)

# Utiliza la función merge para concatenar los DataFrames por 'business_id'
df_ML_Totales = df_business_yelp_ML.merge(df_reviews_totales, on='business_id')

# Resultado
df_ML_Totales.shape

(14378, 41)

In [91]:
df_ML_Totales.isna().sum()

business_id                        0
name                               0
address                            0
city                               0
state                              0
latitude                           0
longitude                          0
stars                              0
review_count                       0
Categorias                         2
SubCategorias                      0
RestaurantsTakeOut               655
BusinessAcceptsCreditCards      1431
RestaurantsDelivery             1532
OutdoorSeating                  3424
BikeParking                     2932
RestaurantsReservations         3938
WiFi                            3793
HasTV                           4053
GoodForKids                     4249
RestaurantsGoodForGroups        4416
Monday                             0
Tuesday                            0
Wednesday                          0
Thursday                           0
Friday                             0
Saturday                           0
S

In [92]:
df_ML_Totales.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'latitude',
       'longitude', 'stars', 'review_count', 'Categorias', 'SubCategorias',
       'RestaurantsTakeOut', 'BusinessAcceptsCreditCards',
       'RestaurantsDelivery', 'OutdoorSeating', 'BikeParking',
       'RestaurantsReservations', 'WiFi', 'HasTV', 'GoodForKids',
       'RestaurantsGoodForGroups', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Saturday', 'Sunday', 'Total_UnaEstrella',
       'Total_DosEstrellas', 'Total_TresEstrellas', 'Total_CuatroEstrellas',
       'Total_CincoEstrellas', 'Total_Estrellas', 'Total_Sentimientos',
       'Total_Sentimientos_Positivos', 'Total_Sentimientos_Neutros',
       'Total_Sentimientos_Negativos', 'Total_useful', 'Total_funny',
       'Total_cool'],
      dtype='object')

In [93]:
df_ML_Totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14378 entries, 0 to 14377
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   business_id                   14378 non-null  object 
 1   name                          14378 non-null  object 
 2   address                       14378 non-null  object 
 3   city                          14378 non-null  object 
 4   state                         14378 non-null  object 
 5   latitude                      14378 non-null  float64
 6   longitude                     14378 non-null  float64
 7   stars                         14378 non-null  float64
 8   review_count                  14378 non-null  int64  
 9   Categorias                    14376 non-null  object 
 10  SubCategorias                 14378 non-null  object 
 11  RestaurantsTakeOut            13723 non-null  object 
 12  BusinessAcceptsCreditCards    12947 non-null  object 
 13  R

### Uniendo DataFrame de df_business y df_Reviews, con df_tip por Negocio (Promedios)

In [94]:
print(f"Dimensiones Dataset df_ML_promedios (Business y Review): {df_ML_Totales.shape}")
print(f"Dimensiones Dataset df_tip_promedio: {df_tip_totales.shape}")

Dimensiones Dataset df_ML_promedios (Business y Review): (14378, 41)
Dimensiones Dataset df_tip_promedio: (55899, 5)


In [95]:
df_tip_totales.columns

Index(['business_id', 'Total_SentimientosTip',
       'Total_SentimientosTip_Positivos', 'Total_SentimientosTip_Neutros',
       'Total_SentimientosTip_Negativos'],
      dtype='object')

In [96]:
### Uniendo DataFrame de df_business y df_Reviews con df_tip por Negocio (Promedios)

# Utiliza la función merge para concatenar los DataFrames por 'business_id'
df_ML_Totales = df_ML_Totales.merge(df_tip_totales, on='business_id')

# Resultado
print(f"Dimensiones Dataset df_ML_promedios (Business, Review y Tip): {df_ML_Totales.shape}")

Dimensiones Dataset df_ML_promedios (Business, Review y Tip): (10171, 45)


In [97]:
### Validando Dimensiones Resultante
df_ML_Totales.shape

(10171, 45)

In [98]:
df_ML_Totales.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'latitude',
       'longitude', 'stars', 'review_count', 'Categorias', 'SubCategorias',
       'RestaurantsTakeOut', 'BusinessAcceptsCreditCards',
       'RestaurantsDelivery', 'OutdoorSeating', 'BikeParking',
       'RestaurantsReservations', 'WiFi', 'HasTV', 'GoodForKids',
       'RestaurantsGoodForGroups', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Saturday', 'Sunday', 'Total_UnaEstrella',
       'Total_DosEstrellas', 'Total_TresEstrellas', 'Total_CuatroEstrellas',
       'Total_CincoEstrellas', 'Total_Estrellas', 'Total_Sentimientos',
       'Total_Sentimientos_Positivos', 'Total_Sentimientos_Neutros',
       'Total_Sentimientos_Negativos', 'Total_useful', 'Total_funny',
       'Total_cool', 'Total_SentimientosTip',
       'Total_SentimientosTip_Positivos', 'Total_SentimientosTip_Neutros',
       'Total_SentimientosTip_Negativos'],
      dtype='object')

In [99]:
df_ML_Totales.isna().sum()

business_id                           0
name                                  0
address                               0
city                                  0
state                                 0
latitude                              0
longitude                             0
stars                                 0
review_count                          0
Categorias                            0
SubCategorias                         0
RestaurantsTakeOut                  308
BusinessAcceptsCreditCards          765
RestaurantsDelivery                 850
OutdoorSeating                     1857
BikeParking                        1677
RestaurantsReservations            2283
WiFi                               2027
HasTV                              2254
GoodForKids                        2493
RestaurantsGoodForGroups           2606
Monday                                0
Tuesday                               0
Wednesday                             0
Thursday                              0


In [100]:
df_ML_Totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10171 entries, 0 to 10170
Data columns (total 45 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_id                      10171 non-null  object 
 1   name                             10171 non-null  object 
 2   address                          10171 non-null  object 
 3   city                             10171 non-null  object 
 4   state                            10171 non-null  object 
 5   latitude                         10171 non-null  float64
 6   longitude                        10171 non-null  float64
 7   stars                            10171 non-null  float64
 8   review_count                     10171 non-null  int64  
 9   Categorias                       10171 non-null  object 
 10  SubCategorias                    10171 non-null  object 
 11  RestaurantsTakeOut               9863 non-null   object 
 12  BusinessAcceptsCre

### Guardando Datasets Consolidado con Business, Review y Tip por Promedios

In [26]:
### Respaldando archivo en formato 'PARQUET'
Ubicacion = "gs://gmy/Datasets_ML"
NombreDelArchivo = 'df_ML_Totales_Reducido'
Extension = '.parquet'
file = os.path.join(Ubicacion, NombreDelArchivo) + Extension
df_ML_promedios.to_parquet(file)

Validado que se guardo el Dataset para el modelo del ML df_ML_promedios.<Br> Sus dimensiones son: (49334, 39)

### 2) Consolidando Datasets por Promedios

In [32]:
### Respaldando archivo en formato 'PARQUET'
Ubicacion = "gs://gmy/Datasets_ML"
NombreDelArchivo = 'df_ML_Promedios'
Extension = '.parquet'
file = os.path.join(Ubicacion, NombreDelArchivo) + Extension
df_ML_totales.to_parquet(file)

### Fin